#### One of the possible dataset in HF with the name WNC is:https://huggingface.co/datasets/reza-alipour/WNC/tree/main/data

#### It is a gated repo, for which i have requested access. But not sure if it is the right one

#### So following CoEDIT, I am using there script as is to download and store the dataset 

https://github.com/facebookresearch/EditEval/blob/main/src/processors/wnc.py


https://github.com/facebookresearch/EditEval/blob/main/src/utils.py

In [ ]:
import os
import urllib
import pandas as pd
from datasets import DatasetDict, Dataset

In [2]:
def download_url(url, root, filename=None):
    """Download a file from a url and place it in root.
    Args:
        url (str): URL to download file from
        root (str): Directory to place downloaded file in
        filename (str, optional): Name to save the file under. If None, use the basename of the URL
    """

    root = os.path.expanduser(root)
    if not filename:
        filename = os.path.basename(url)
    fpath = os.path.join(root, filename)

    os.makedirs(root, exist_ok=True)

    try:
        print("Downloading " + url + " to " + fpath)
        urllib.request.urlretrieve(url, fpath)
    except (urllib.error.URLError, IOError) as e:
        if url[:5] == "https":
            url = url.replace("https:", "http:")
            print("Failed download. Trying https -> http instead." " Downloading " + url + " to " + fpath)
            urllib.request.urlretrieve(url, fpath)

In [3]:
def download_and_process(url, download_dir, filename):
#     Lets first download the data
    if not os.path.exists(download_dir):
        download_url(url, download_dir, filename)
        os.system(f"unzip {os.path.join(download_dir, 'bias-corpus')} -d {download_dir}")
        os.system(f"rm {os.path.join(download_dir, 'bias-corpus')}")
        
        
    original_columns = [
            "id",
            "src_tok",
            "tgt_tok",
            "input",
            "edits",
            "src_POS_tags",
            "tgt_parse_tags",
        ]

    dfs = []
    splits= ["train", "dev", "test"]
    for split in splits:
        full_path = os.path.join(download_dir, "bias_data", "WNC", "biased.word." + split)
        df = pd.read_csv(full_path, sep="\t", names=original_columns)
        df["id"] = df["id"].apply(lambda x: "wnc-" + split + "-" + str(x))
        df["wiki_id"] = df["id"].apply(lambda x: x)
        dfs.append(df)

    train_dataset = Dataset.from_pandas(dfs[0]) 
    dev_dataset = Dataset.from_pandas(dfs[1])
    test_dataset = Dataset.from_pandas(dfs[2])
    ds = DatasetDict(
        {
            "train": train_dataset,
            "val": dev_dataset, 
            "test": test_dataset
        }
    )

    return ds

In [4]:
url="http://bit.ly/bias-corpus"
download_dir='./wnc'
filename='bias-corpus.zip'
ds= download_and_process(url, download_dir, filename)

In [5]:
ds

DatasetDict({
    train: Dataset({
        features: ['id', 'src_tok', 'tgt_tok', 'input', 'edits', 'src_POS_tags', 'tgt_parse_tags', 'wiki_id'],
        num_rows: 53803
    })
    val: Dataset({
        features: ['id', 'src_tok', 'tgt_tok', 'input', 'edits', 'src_POS_tags', 'tgt_parse_tags', 'wiki_id'],
        num_rows: 700
    })
    test: Dataset({
        features: ['id', 'src_tok', 'tgt_tok', 'input', 'edits', 'src_POS_tags', 'tgt_parse_tags', 'wiki_id'],
        num_rows: 1000
    })
})

In [6]:
dir_path='/scratch3/workspace/wenlongzhao_umass_edu-reason/dev_kedar/Small-LLM-Reasoning/datasets/'
ds.save_to_disk(dir_path+'neutralization')

Saving the dataset (0/1 shards):   0%|          | 0/53803 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/700 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/1000 [00:00<?, ? examples/s]

### Spliting train set into feedback and train set

In [1]:
from datasets import load_from_disk

In [2]:
train=load_from_disk('../datasets/neutralization/train/')
train

Dataset({
    features: ['id', 'src_tok', 'tgt_tok', 'input', 'edits', 'src_POS_tags', 'tgt_parse_tags', 'wiki_id'],
    num_rows: 53803
})

In [6]:
import json
task='neutralization'
task_prompt_path=f'../prompts/{task}.json'
with open(task_prompt_path) as fp:
    task_prompt = json.load(fp)
task_prompt

{'task_prompt': 'Given an input sentence, reason and replace the word with subjective bias to a word with neutral point of view. Consider the following types of biases:\n1. framing biases use subjective words linked with a particular point of view (e.g. using words like best or deepest or using pilfered from instead of based on);\n2. epistemological biases are linguistic features that subtly (often via presupposition) modify the believability of a proposition;\n3. demographic biases are texts with presuppositions about particular genders, races, or other demographic categories (e.g. presupposing that all programmers are male).\n\nYour response should end with "The neutralized text is [answer]" where [answer] is the neutralized version of the input sentence.',
 'few_shot': [{'id': 'wnc-train-131576896',
   'input': 'gender-neutral pronouns used in ancient english',
   'reference': 'gender-neutral pronouns used in middle english',
   'rationale': 'The word ancient should be replaced by m

In [9]:
few_shot_ids=[]
for i in task_prompt['few_shot']:
    few_shot_ids.append(i['id'])
few_shot_ids


['wnc-train-131576896',
 'wnc-train-169899659',
 'wnc-train-712554371',
 'wnc-train-165621694',
 'wnc-train-417594576',
 'wnc-train-207072758',
 'wnc-train-692108570',
 'wnc-train-693565497',
 'wnc-train-70218028',
 'wnc-train-879288389',
 'wnc-train-88741599']

In [11]:
train=train.filter(lambda row:row['wiki_id'] not in few_shot_ids)
train

Filter:   0%|          | 0/53803 [00:00<?, ? examples/s]

Dataset({
    features: ['id', 'src_tok', 'tgt_tok', 'input', 'edits', 'src_POS_tags', 'tgt_parse_tags', 'wiki_id'],
    num_rows: 53792
})

In [12]:
import random

feedback=train.shuffle(42).select(range(2500))
feedback

Dataset({
    features: ['id', 'src_tok', 'tgt_tok', 'input', 'edits', 'src_POS_tags', 'tgt_parse_tags', 'wiki_id'],
    num_rows: 2500
})

In [13]:
feedback.save_to_disk('../datasets/neutralization/feedback/')

Saving the dataset (0/1 shards):   0%|          | 0/2500 [00:00<?, ? examples/s]